In [1]:
# Here are some imports that are used along this notebook
import math
import itertools
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from collections import OrderedDict
import glob
%matplotlib inline
gt0 = time()

In [2]:
train20_nsl_kdd_dataset_path = "C:/Users/user/MSDS/Capstone/NSL_KDD-master/NSL_KDD-master/KDDTrain+.txt"
#train_nsl_kdd_dataset_path = "C:/Users/user/MSDS/Capstone/NSL_KDD-master/NSL_KDD-master/KDDTrain+.txt"
test_nsl_kdd_dataset_path = "C:/Users/user/MSDS/Capstone/NSL_KDD-master/NSL_KDD-master/KDDTest+.txt"

col_names = np.array(["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels","attrib43"])


nominal_inx = [1, 2, 3]
binary_inx = [6, 11, 13, 14, 20, 21]
numeric_inx = list(set(range(41)).difference(nominal_inx).difference(binary_inx))

nominal_cols = col_names[nominal_inx].tolist()
binary_cols = col_names[binary_inx].tolist()
numeric_cols = col_names[numeric_inx].tolist()
print(nominal_cols)
print(binary_cols)

# Dictionary that contains mapping of various attacks to the four main categories
attack_dict = {
    'normal': 'normal',
    
    'back': 'DoS',
    'land': 'DoS',
    'neptune': 'DoS',
    'pod': 'DoS',
    'smurf': 'DoS',
    'teardrop': 'DoS',
    'mailbomb': 'DoS',
    'apache2': 'DoS',
    'processtable': 'DoS',
    'udpstorm': 'DoS',
    
    'ipsweep': 'Probe',
    'nmap': 'Probe',
    'portsweep': 'Probe',
    'satan': 'Probe',
    'mscan': 'Probe',
    'saint': 'Probe',

    'ftp_write': 'R2L',
    'guess_passwd': 'R2L',
    'imap': 'R2L',
    'multihop': 'R2L',
    'phf': 'R2L',
    'spy': 'R2L',
    'warezclient': 'R2L',
    'warezmaster': 'R2L',
    'sendmail': 'R2L',
    'named': 'R2L',
    'snmpgetattack': 'R2L',
    'snmpguess': 'R2L',
    'xlock': 'R2L',
    'xsnoop': 'R2L',
    'worm': 'R2L',
    
    'buffer_overflow': 'U2R',
    'loadmodule': 'U2R',
    'perl': 'U2R',
    'rootkit': 'U2R',
    'httptunnel': 'U2R',
    'ps': 'U2R',    
    'sqlattack': 'U2R',
    'xterm': 'U2R'
}

['protocol_type', 'service', 'flag']
['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login', 'is_guest_login']


In [3]:
def _label2(x):
    if x['labels'] == 'normal':
        return 'normal'
    else:
        return 'attack'

def returnvalue(x):
    return attack_dict.get(x['labels'])

In [4]:
df_kdd_dataset_train = pd.read_csv(train20_nsl_kdd_dataset_path, index_col=None, names=col_names)
df_kdd_dataset_train['label2'] = df_kdd_dataset_train.apply(_label2,axis=1)
df_kdd_dataset_train['label3'] = df_kdd_dataset_train.apply(returnvalue,axis=1)

In [5]:
df_kdd_dataset_test = pd.read_csv(test_nsl_kdd_dataset_path, index_col=None, names=col_names)
df_kdd_dataset_test['label2'] = df_kdd_dataset_test.apply(_label2,axis=1)
df_kdd_dataset_test['label3'] = df_kdd_dataset_test.apply(returnvalue,axis=1)

In [6]:
print(df_kdd_dataset_train.shape)
print(df_kdd_dataset_test.shape)

(125973, 45)
(22544, 45)


In [7]:
df_concatenate = [df_kdd_dataset_train,df_kdd_dataset_test]
df_kdd_entire_set = pd.concat(df_concatenate)
print(df_kdd_entire_set.shape)

(148517, 45)


In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_kdd_entire_set["protocol_type"] = le.fit_transform(df_kdd_entire_set["protocol_type"])
df_kdd_entire_set["service"] = le.fit_transform(df_kdd_entire_set["service"])
df_kdd_entire_set["flag"] = le.fit_transform(df_kdd_entire_set["flag"])

In [9]:
import csv
csvfile = "C:/Users/user/MSDS/Capstone/NSL_KDD-master/NSL_KDD-master/output4.txt"

#Assuming res is a flat list
df_kdd_entire_set.to_csv(csvfile)

In [10]:
def one_hot_encode(x, n_classes):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
     """
    return np.eye(n_classes)[x]


In [11]:
y_label2 = df_kdd_entire_set['label2']
le.fit(y_label2)
y_label2 = le.transform(y_label2)
print(y_label2)
y_label2 = one_hot_encode(y_label2,2)
print(y_label2)

[1 1 0 ..., 0 1 0]
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]


In [12]:
y_label3 = df_kdd_entire_set['label3']
le.fit(y_label3)
y_label3 = le.transform(y_label3)
print(y_label3)
y_label3 = one_hot_encode(y_label3,5)
print(y_label3)

[4 4 0 ..., 0 4 1]
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.]]


In [13]:
df_kdd_entire_set.drop('labels', axis=1, inplace=True)
df_kdd_entire_set.drop('attrib43',axis=1,inplace=True)
df_kdd_entire_set.drop('label2',axis=1,inplace=True)
df_kdd_entire_set.drop('label3',axis=1,inplace=True)

#enc = preprocessing.OneHotEncoder(categorical_features=[1, 2, 3])

#enc.fit(df_kdd_entire_set)

# 3. Transform
#onehotlabels_train_test = enc.transform(df_kdd_entire_set).toarray()
#print(onehotlabels_train_test.shape)

In [14]:
train_size = df_kdd_dataset_train.shape[0]
dff_kdd_train = df_kdd_entire_set[0:train_size]
dff_kdd_test = df_kdd_entire_set[train_size:df_kdd_entire_set.shape[0]]
print(dff_kdd_train.shape)
print(dff_kdd_test.shape)

(125973, 41)
(22544, 41)


In [15]:
import csv
csvfile = "C:/Users/user/MSDS/Capstone/NSL_KDD-master/NSL_KDD-master/output.txt"

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in dff_kdd_train:
        writer.writerow([val])   

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train_transform = scaler.fit_transform(dff_kdd_train)
df_test_transform = scaler.fit_transform(dff_kdd_test)

In [18]:
print(df_train_transform.shape)

(125973, 41)


In [19]:
y_label2_train = y_label2[0:train_size]
y_label2_test = y_label2[train_size:df_kdd_entire_set.shape[0]]

In [20]:
print(y_label2_train)
print(y_label2_test)
df_train_transform_with_label=np.c_[df_train_transform,y_label2_train]
df_test_transform_with_label=np.c_[df_test_transform,y_label2_test]
print(df_train_transform_with_label.shape)
print(df_test_transform_with_label.shape)

[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 ..., 
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]]
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 ..., 
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
(125973, 43)
(22544, 43)


In [21]:
def get_next_batch(i,batch_size,dataset):
    start = i*batch_size
    end = (i+1)*batch_size
    return dataset[start:end]

In [22]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 1000
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 20 # 1st layer num features
#n_hidden_2 = 25 # 2nd layer num features
n_input = 41 # Number of features

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])
autoencoder_op = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    #'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    #'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    #'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    #'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    #layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    print('encoder')
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    #layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    print('decoder')
    return layer_1

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
    
model_path = "C:\\users\\user\MSDS\\Capstone\\Models"

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(df_train_transform.shape[0]/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = get_next_batch(i,batch_size,df_train_transform)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs}) 
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1),
                  "cost=", "{:.9f}".format(c))            
    print("Optimization Finished!")
    #c2,dec_op2 = sess.run([cost,decoder_op], feed_dict={X: df_train_transform}) 
    save_path = saver.save(sess,model_path)
    # model has been trained pass the training data with labels
    cost_train,encoder_op_train_wo_label = sess.run([cost,encoder_op], feed_dict={X: df_train_transform}) 
    print(cost_train)
    cost_test,encoder_op_test_wo_label = sess.run([cost,encoder_op], feed_dict={X: df_test_transform}) 
    print(cost_test)

encoder
decoder
Epoch: 0001 cost= 0.129345387
Epoch: 0002 cost= 0.008430441
Epoch: 0003 cost= 0.002446069
Epoch: 0004 cost= 0.001504005
Epoch: 0005 cost= 0.001232216
Epoch: 0006 cost= 0.001076626
Epoch: 0007 cost= 0.001002824
Epoch: 0008 cost= 0.000896854
Epoch: 0009 cost= 0.000865728
Epoch: 0010 cost= 0.000826851
Epoch: 0011 cost= 0.000828474
Epoch: 0012 cost= 0.000899145
Epoch: 0013 cost= 0.000778568
Epoch: 0014 cost= 0.000793496
Epoch: 0015 cost= 0.000766689
Epoch: 0016 cost= 0.000777874
Epoch: 0017 cost= 0.000733435
Epoch: 0018 cost= 0.000735751
Epoch: 0019 cost= 0.000706409
Epoch: 0020 cost= 0.000710744
Epoch: 0021 cost= 0.000742098
Epoch: 0022 cost= 0.000701615
Epoch: 0023 cost= 0.000707584
Epoch: 0024 cost= 0.000678231
Epoch: 0025 cost= 0.000693615
Epoch: 0026 cost= 0.000690759
Epoch: 0027 cost= 0.000684252
Epoch: 0028 cost= 0.000684984
Epoch: 0029 cost= 0.000666772
Epoch: 0030 cost= 0.000667041
Epoch: 0031 cost= 0.000646706
Epoch: 0032 cost= 0.000659855
Epoch: 0033 cost= 0.0006

In [23]:
# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 1000
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 10 # 1st layer num features
#n_hidden_2 = 25 # 2nd layer num features
n_input = 20 # Number of features

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])
autoencoder_op = tf.placeholder("float", [None, n_input])

weights_S2 = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    #'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    #'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases_S2 = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    #'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    #'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder_S2(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights_S2['encoder_h1']),biases_S2['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    #layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    print('encoder')
    return layer_1


# Building the decoder
def decoder_S2(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights_S2['decoder_h1']),biases_S2['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    #layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    print('decoder')
    return layer_1

# Construct model
encoder_op = encoder_S2(X)
decoder_op = decoder_S2(encoder_op)
    
model_path = "C:\\users\\user\MSDS\\Capstone\\Models"

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(df_train_transform.shape[0]/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = get_next_batch(i,batch_size,encoder_op_train_wo_label)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs}) 
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1),
                  "cost=", "{:.9f}".format(c))            
    print("Optimization Finished!")
    #c2,dec_op2 = sess.run([cost,decoder_op], feed_dict={X: df_train_transform}) 
    save_path = saver.save(sess,model_path)
    # model has been trained pass the training data with labels
    cost_train,encoder_op2_train_wo_label = sess.run([cost,encoder_op], feed_dict={X: encoder_op_train_wo_label}) 
    print(cost_train)
    cost_test,encoder_op2_test_wo_label = sess.run([cost,encoder_op], feed_dict={X: encoder_op_test_wo_label}) 
    print(cost_test)

encoder
decoder
Epoch: 0001 cost= 0.081370324
Epoch: 0002 cost= 0.016366350
Epoch: 0003 cost= 0.008956696
Epoch: 0004 cost= 0.007210674
Epoch: 0005 cost= 0.006335004
Epoch: 0006 cost= 0.005588522
Epoch: 0007 cost= 0.005190033
Epoch: 0008 cost= 0.005108485
Epoch: 0009 cost= 0.004837396
Epoch: 0010 cost= 0.004741573
Epoch: 0011 cost= 0.004685895
Epoch: 0012 cost= 0.004579602
Epoch: 0013 cost= 0.004525463
Epoch: 0014 cost= 0.004457514
Epoch: 0015 cost= 0.004291519
Epoch: 0016 cost= 0.004184833
Epoch: 0017 cost= 0.004149614
Epoch: 0018 cost= 0.004133228
Epoch: 0019 cost= 0.004233181
Epoch: 0020 cost= 0.004048833
Epoch: 0021 cost= 0.003995394
Epoch: 0022 cost= 0.003888939
Epoch: 0023 cost= 0.004012542
Epoch: 0024 cost= 0.004017576
Epoch: 0025 cost= 0.003990760
Epoch: 0026 cost= 0.003833093
Epoch: 0027 cost= 0.003887956
Epoch: 0028 cost= 0.003836720
Epoch: 0029 cost= 0.003866416
Epoch: 0030 cost= 0.003661667
Epoch: 0031 cost= 0.003752113
Epoch: 0032 cost= 0.003746479
Epoch: 0033 cost= 0.0037

In [24]:
print(encoder_op2_train_wo_label.shape)
print(encoder_op2_test_wo_label.shape)

(125973, 10)
(22544, 10)


In [25]:
def get_next_batch_label(i,batch_size,dataset):
    start = i*batch_size
    end = (i+1)*batch_size
    return dataset[start:end]

In [28]:
def get_next_batch_sftmax(i,batch_size):
    start = i*batch_size
    end = (i+1)*batch_size
    return encoder_op2_train_wo_label[start:end,0:10],df_train_transform_with_label[start:end,41:43]

In [29]:
## Now we build the softmax regressor

# Parameters
learning_rate = 0.1
training_epochs = 1000
batch_size = 1000
display_step = 10

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 10]) # 118 features
y = tf.placeholder(tf.float32, [None, 2]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([10, 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(df_train_transform_with_label.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = get_next_batch_sftmax(i,batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    print("accuracy", sess.run(accuracy, feed_dict={x: encoder_op2_train_wo_label[:,0:10], y: df_train_transform_with_label[:,41:43]}))
    #prediction=tf.argmax(y,1)
    #print ("predictions", prediction.eval(feed_dict={x: df_train_transform_with_label[0:118]}, session=sess))
    print("accuracy", sess.run(accuracy, feed_dict={x: encoder_op2_test_wo_label[:,0:10], y: df_test_transform_with_label[:,41:43]}))
    #print("accuracy", sess.run(accuracy, feed_dict={x: df_train_transform_with_label[:,0:118], y: df_train_transform_with_label[:,118:120]}))
    

Epoch: 0010 cost= 0.184840858
Epoch: 0020 cost= 0.175871925
Epoch: 0030 cost= 0.173077309
Epoch: 0040 cost= 0.171578419
Epoch: 0050 cost= 0.170582937
Epoch: 0060 cost= 0.169857987
Epoch: 0070 cost= 0.169304316
Epoch: 0080 cost= 0.168868759
Epoch: 0090 cost= 0.168519005
Epoch: 0100 cost= 0.168233908
Epoch: 0110 cost= 0.167998927
Epoch: 0120 cost= 0.167803657
Epoch: 0130 cost= 0.167640408
Epoch: 0140 cost= 0.167503299
Epoch: 0150 cost= 0.167387780
Epoch: 0160 cost= 0.167290191
Epoch: 0170 cost= 0.167207603
Epoch: 0180 cost= 0.167137613
Epoch: 0190 cost= 0.167078230
Epoch: 0200 cost= 0.167027813
Epoch: 0210 cost= 0.166984965
Epoch: 0220 cost= 0.166948550
Epoch: 0230 cost= 0.166917576
Epoch: 0240 cost= 0.166891232
Epoch: 0250 cost= 0.166868808
Epoch: 0260 cost= 0.166849729
Epoch: 0270 cost= 0.166833481
Epoch: 0280 cost= 0.166819649
Epoch: 0290 cost= 0.166807867
Epoch: 0300 cost= 0.166797835
Epoch: 0310 cost= 0.166789292
Epoch: 0320 cost= 0.166782015
Epoch: 0330 cost= 0.166775813
Epoch: 034

In [ ]:
print(dec_op_test_label[1])

In [ ]:
def get_next_batch_sftmax_wostl(i,batch_size):
    start = i*batch_size
    end = (i+1)*batch_size
    return df_train_transform_with_label[start:end,0:41],df_train_transform_with_label[start:end,41:43]

In [ ]:
## Now we build the softmax regressor

# Parameters
learning_rate = 0.3
training_epochs = 1000
batch_size = 1000
display_step = 10

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 41]) # 118 features
y = tf.placeholder(tf.float32, [None, 2]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([41, 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(df_train_transform_with_label.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = get_next_batch_sftmax_wostl(i,batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    print("accuracy", sess.run(accuracy, feed_dict={x: df_train_transform_with_label[:,0:41], y: df_train_transform_with_label[:,41:43]}))
    #prediction=tf.argmax(y,1)
    #print ("predictions", prediction.eval(feed_dict={x: df_train_transform_with_label[0:118]}, session=sess))
    print("accuracy", sess.run(accuracy, feed_dict={x: df_test_transform_with_label[:,0:41], y: df_test_transform_with_label[:,41:43]}))
    #print("accuracy", sess.run(accuracy, feed_dict={x: df_train_transform_with_label[:,0:118], y: df_train_transform_with_label[:,118:120]}))
    